## ENPH 353 Lecture 2 - An introduction to modeling and non-linear curve fitting

Version 1.1 August 21 2018, Rob Knobel

## Names: ______________________  and ______________ and _____________ and _____________

This python notebook contains skeleton code of how to do a non-linear curve fit using standard python packages (matplotlib, scipy.optimize).  You can re-use this code later in the term if it is helpful.  At the end of the class period, you will upload a pdf printout of this workbook for grading.  Details are below.

The first few cells load in the required packages for python.  To evaluate each cell, select it with your mouse then click **shift-enter**.

In [ ]:
# Brings in the numpy package, giving numerical functions to python.  All calls to functions in the package are prefaced by np.
import numpy as np

In [ ]:
# Brings in the matplotlib graphics package.  All calls to it prefaced by plt.
import matplotlib as plt
 #  This makes the plots from matplotlib appear in the notebook.
%matplotlib inline

In [ ]:
# More importing of the curve_fit package and pylab has plotting tools.
from scipy.optimize import curve_fit
from pylab import *

Next, load the data from the text file.  The data is stored as time (in milliseconds), signal (in volts) and uncertainty in the signal (in volts).

In [ ]:
data = np.genfromtxt('curve fit exercise.csv',delimiter=',', skip_header=1)

In [ ]:
print(data)

In [ ]:
# Break up the multi-dimensional data into three vectors for time, signal and uncertainty.
time = data[:,0]
signal = data[:,1]
uncertainty = data[:,2]

We can plot using the matplotlib package for python.  Using the tips available at, for instance, https://matplotlib.org/examples/showcase/anatomy.html

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))   #  A sigle plot 8cm square
ax.set_title('Test data for fitting')    #  Plot title
ax.set_ylabel('Signal (V)')              #  y-axis label
ax.set_xlabel('Time (ms)')               #  x-axis label 
ax.errorbar(time,signal,uncertainty,fmt='o',color='r')   # errorbar plot with circle symbols in red
plt.show()                               #  Show the plot below

The data will be fit to the function

$d(t) = A(1+B \cos \omega t) e^{-t^2/2\tau^2} +C$

where the fitting parameters are $A$, $B$, $C$, $\omega$ and $\tau$.  The next cell defines the fitting function.

In [ ]:
# define the fitting function
def fit_func(t,a,b,c,omega,tau):
    return a*(1+b*np.cos(omega*t))*np.exp(-t**2.0/(2.0*tau**2.0))+c
# count the number of times we run the fit
iteration = 1

## Exercise:
Estimate starting parameters for the fit (given in the **start** array below) that give a good fit. Rather than replacing the numbers, please copy the next cell over each time you change the fit.  

It is best to understand how the fitting function behaves, and have initial guesses that are somewhat close to the correct values.  Feel free to use cells in the notebook to compute this.

How can you tell if the fit is "good"?  How good is it?

In [ ]:
# Starting parameter values:  A, B, C, omega, tau
#  THIS IS WHAT WE'LL CHANGE TO START THE FIT AT A DIFFERENT POINT
start = np.array([1.0, 1.0, 1.0, 1.0, 1.0])


# Perform the fit
popt,pcov=curve_fit(fit_func,time,signal,p0=start,sigma=uncertainty,absolute_sigma=True)

#Plot the data and fit
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,figsize=(8, 4))

ax1.errorbar(time,signal,uncertainty,fmt='o')
x = np.linspace(0,40,1000)
ax1.plot(x,fit_func(x,popt[0],popt[1],popt[2],popt[3],popt[4]),'r')
title = 'Test fit number ' + str(iteration)
ax1.set_title(title)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal (V)')

# Plot the residuals

ax2.errorbar(time,signal-fit_func(time,popt[0],popt[1],popt[2],popt[3],popt[4]),uncertainty,fmt='o')
title = 'Residuals for fit number ' + str(iteration)
ax2.set_title(title)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Signal difference (V)')
plt.subplots_adjust(wspace=0.35)

# Show the equation
text = r'$d(t) = A(1 + B \cos \omega t) e^{-\frac{t^2}{2 \tau^2}} + C$'
ax = plt.gca()
ax.text(1.2,0.5,text,transform=ax.transAxes,fontsize=16)

plt.show()
iteration = iteration + 1

# Print out the coefficients and the 1-sigma uncertainty on the coefficients.  The uncertainty on each coefficient is
#    given by the square root of the diagonal of the covariance matrix pcov.

coefficient_err = np.sqrt(np.diag(pcov))
coefficients = popt
print("Start A =\t", start[0],"\t\t, Final A =\t",coefficients[0]," +/- ",coefficient_err[0])
print("Start B =\t", start[1],"\t\t, Final B =\t",coefficients[1]," +/- ",coefficient_err[1])
print("Start C =\t", start[2],"\t\t, Final C =\t",coefficients[2]," +/- ",coefficient_err[2])
print("Start omega =\t",start[3],"\t\t, Final omega =\t",coefficients[3]," +/- ",coefficient_err[3])
print("Start tau =\t",start[4], "\t\t, Final tau =\t",coefficients[4]," +/- ",coefficient_err[4])

# Calculate the Chi-squared goodness of fit statistic, and the reduced chi squared statistic.
fitted = fit_func(time, *popt)
residuals = signal - fitted
chisq = np.sum((residuals/uncertainty)**2)
degree_freedom = len(time) - len(popt)
print("\n chisq =",chisq,"\t df =",degree_freedom,"\t\t reduced chisq = ",chisq/degree_freedom)

In [ ]:
COPY THE PREVIOUS CELL HERE AND VARY THE STARTING PARAMETERS.

### Answer the following questions and do the following exercises (if time permits):
* What is the smallest value of $\chi^2$ you obtained?
* Are the quoted uncertainties in the parameters smallest for this fit?  If the uncertainties aren't the smallest for this fit, what does that mean?
* In a later cell there is a function which looks at the quality of the fit while changing one parameter. We fix all the parameters to the best fit values **except** $\omega$ and do a loop with $\omega$ varying from .05 to 3.95.  Plot $\chi^2$ as a function of $\omega$.  Describe the fit.  Where is the best fit value?  Describe what is happening in the minimum around 1.4.

In [ ]:
# Take the previously fit parameters but vary the frequency omega.
best = popt
omega_range = np.linspace(.05, 3.95, num=179)

# Loop over the omega_range values
chi_range = []


for om in omega_range:
    popt[3]=om
    fitted = fit_func(time, *popt)
    residuals = signal - fitted               # Calculate the residuals for best fit with varying omega
    chisq = np.sum((residuals/uncertainty)**2)  # Calculate Chi squared for that fit
    degree_freedom = len(time) - len(popt)
#    print("Omega =", om,"\t chisq =",chisq,"\t df =",degree_freedom,"\t\t reduced chisq = ",chisq/degree_freedom)
    chi_range=np.append(chi_range, chisq/degree_freedom)    # Build an array of the reduced Chi squared for each omega

#Plot the reduced chi squared vs. omega    
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title('Quality of Fit')
ax.set_ylabel('Reduced Chi Squared')
ax.set_xlabel('Omega (rad/s)')
ax.plot(omega_range,chi_range)
plt.show()